In [ ]:
import json
from NumpyModel import Model
from Layers import InputLayer, Conv2D, Dense, DephtwiseConv2D, Flatten
from Activations.ActivationFactory import activation_from_name
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

np.random.seed(2)

# create numpy model

In [ ]:
classifier_model_path = r"saved_models/classifier_v2/"

model = Model()
model.load_model(classifier_model_path)

for l in model.layers: print(l)

# test numpy model

In [ ]:
input_test = np.ones(shape=[1, 256, 256, 3])

timelist = []
runnings = 10

for i in range(runnings): 
  o_ref_model, time_per_layer = model(input_test, training=False)
  timelist.append(time_per_layer)

print("model test passed...")
print(o_ref_model)

In [ ]:
mean_elapsed_time_per_layer = {}

for r in range(runnings):
    for l in timelist[r]["time_per_layer"]:
        if l not in mean_elapsed_time_per_layer:
            mean_elapsed_time_per_layer[l] = 0        
        mean_elapsed_time_per_layer[l] += (timelist[r]["time_per_layer"][l]["elapsedtime"] / runnings)

total_elapsed = 0
for k, v in mean_elapsed_time_per_layer.items():
    total_elapsed += v
    print(k, v)

print("total_elapsed", total_elapsed)

In [ ]:


img = cv.imread("resized_dataset_texturas_v2/tijolo_alv/img_100.jpeg")
img = cv.cvtColor(img, cv.COLOR_BGR2RGB) / 255
plt.imshow(img)

# predict
output = model(img[np.newaxis, ...])

print(output)
